# 프롬프트(Prompt)
`02_prompt.ipynb``

- LLM한테 주는 입력(지시, 맥락, 기억)
    - 지시: '~해줘'
    - 맥락(Context): 현재 지시를 위해 제공하는 추가 정보
    - 기억(Memory): 지금까지 헀던 대화의 내용

In [1]:
# 이건 원래 해야돼요~
from dotenv import load_dotenv

load_dotenv()

True

## `Prompt Template``
- 일회성 명령을 낼 때 활용

In [9]:
# LangChain 쓰기 위한 필수 라이브러리
from langchain_core.prompts import PromptTemplate # prompt
from langchain_openai import ChatOpenAI # model(조정할 게 별로 없음)
from langchain_core.output_parsers import StrOutputParser # parser

llm = ChatOpenAI(model='gpt-4.1-nano')

In [6]:
# 추후에 체인에서 {} 내부에 들어갈 말읗 채워줘야 함.
template = 'What is capital city of {Country}?'
prompt = PromptTemplate.from_template(template) # PromptTemplate(input_variables=['Country']

# {} 채우는 방법(We eventually use chain.invoke)
prompt.format(Country = 'Russia')

'What is capital city of Russia?'

In [13]:
chain = prompt | llm | StrOutputParser()

chain.invoke({'Country': '대북조선한미국'})

'"대북조선한미국" appears to be a combination of Korean words: "대북" (regarding North Korea), "조선" (another term for Korea), "한미" (Korea and the United States), and "국" (country). However, this combination does not correspond to the name of an actual country.\n\nIf you are referring to a specific country or entity, please clarify or provide more context. \n\nFor example:\n- North Korea is called 조선 (Choson), and its capital is Pyongyang.\n- South Korea\'s capital is Seoul.\n- The United States\' capital is Washington, D.C.\n\nPlease specify your question so I can assist you better.'

# Partial Variable(부분 변수)

- 프롬프트에서 매개변수 기본값 지정해주기.

In [ ]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template = template,
    input_variables = ['c1'],
    partial_variables= {
        'c2': 'America'
    }
)

prompt.format(c1='Korea')
# prompt.format(c2='Korea')



'Korea과 America의 수도는 각각 어디인가요?'

In [21]:
chain = prompt | llm | StrOutputParser()
chain.invoke({'c1': 'kor', 'c2':'chn'})

'한국(KOR)의 수도는 서울이고, 중국(CHN)의 수도는 베이징입니다.'

In [27]:
from datetime import datetime
print(datetime.now())

prompt = PromptTemplate(
    template = '오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해주세요.',
    input_variables = ['n'],
    partial_variables= {
        'today': lambda : datetime.now().strftime('%b %d'),
    }
)

prompt.format(n=3)

2025-09-01 15:16:10.634218


'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해주세요.'

In [29]:
chain = prompt | llm | StrOutputParser()
chain.invoke({'n': 3})

chain.invoke({'n': 3, 'today': 'Jan 2'})

'2024년 1월 2일 기준으로 오늘(1월 2일) 생일인 유명인 3명은 다음과 같습니다:\n\n1. 폴 뉴먼 (Paul Newman) - 1925년 1월 2일\n2. 맥스웰 디샤 (Maxwell Dorsey) - 1894년 1월 2일\n3. 에드워드 제임스 올멧 (Edward James Olmos) - 1947년 2월 24일 (이 경우 생일이 아니지만, 참고로 제공)\n\n**참고:**  \n- 폴 뉴먼은 미국의 배우이자 영화 제작자로 유명하며, 2024년 현재 생일입니다.  \n- 맥스웰 디샤는 역사적 인물이기 때문에 공개된 정보가 제한적일 수 있습니다.  \n- 거의 정확한 정보를 위해 다시 한 번 확인하는 것이 좋습니다.\n\n혹시 더 궁금한 점이 있으시면 말씀해 주세요!'

## `ChatPromptTemplate`
- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat은 `Role`과 `Message`로 구성됨

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chatprompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chatprompt.format(country = '한국') # 'Human: 한국의 수도는 어디?'(역할이 명시됨)

'Human: 한국의 수도는 어디?'

In [35]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 친절한 AI 어시트트. 이름은 {name}.'),
        ('human', 'Nice to meet you~.'),
        ('ai' , 'what can I help you?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='등신', user_input = 'who 이름지어 you.')
messages_cls = chat_template.format_messages(name='등신', user_input = 'who 이름지어 you.')
print(messages_str) # 사람이 보기 좋은 형태
print(messages_cls) # 컴퓨터가 보기 좋은 형태

System: 당신은 친절한 AI 어시트트. 이름은 등신.
Human: Nice to meet you~.
AI: what can I help you?
Human: who 이름지어 you.
[SystemMessage(content='당신은 친절한 AI 어시트트. 이름은 등신.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Nice to meet you~.', additional_kwargs={}, response_metadata={}), AIMessage(content='what can I help you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='who 이름지어 you.', additional_kwargs={}, response_metadata={})]


In [ ]:
llm.invoke(messages_str).content # 이렇게 주면 안 돼.(human message 한 덩어리로 인지함.)
llm.invoke(messages_cls).content # 얘는 됨.

'저를 "등신"이라고 불러주셔서 감사합니다! 혹시 다른 이름도 지어드릴까요?'

In [41]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': 'supremeT', 'user_input': 'you know who I am?'})

"Nice to meet you! I don't have the ability to know who you are unless you tell me. How can I assist you today?"

## Prompt Engineering

- Zero-shot Prompting
    - 예시 없음 -> 질문 이후 바로 답변
- One-shot Prompting
    - 예시 1개 + 질문 -> 모델이 예시를 모방
- **Few-shot Prompting**
    - 예시 여러 개 + 질문 -> 예시들의 패턴을 일반화

In [42]:
pass

## `Langchain-Hub`
- 다른 사용자의 프롬프트를 받아서 활용할 수 있음.

In [44]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [48]:
# langchain-hub에 등록해보기.
prompt = PromptTemplate.from_template(
    'Stop using AI and play with puppies'
)

hub.push('prompt-hub-test', prompt)

'https://smith.langchain.com/prompts/prompt-hub-test/d96188f1?organizationId=9dc78c9e-094d-477d-84dc-c9f4abab2caf'